In [ ]:
####################################
#ENVIRONMENT SETUP

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py
from tqdm import tqdm

In [ ]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [ ]:
def GetPlottingDirectory(plotFileName, plotType):
    plottingDirectory = mainCodeDirectory=os.path.join(mainDirectory,"Code","PLOTTING")
    
    specificPlottingDirectory = os.path.join(plottingDirectory, plotType, 
                                             f"{ModelData.res}_{ModelData.t_res}_{ModelData.Nz_str}nz")
    os.makedirs(specificPlottingDirectory, exist_ok=True)

    plottingFileName=os.path.join(specificPlottingDirectory, plotFileName)

    return plottingFileName

def SaveFigure(fig,plotType, fileName):
    plotFileName = f"{fileName}_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}.jpg"
    plottingFileName = GetPlottingDirectory(plotFileName, plotType)
    print(f"Saving figure to {plottingFileName}")
    fig.savefig(plottingFileName, dpi=300, bbox_inches='tight')

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [ ]:
#IMPORT FUNCTIONS
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
import FUNCTIONS_Variable_Calculation
from FUNCTIONS_Variable_Calculation import *

In [ ]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="Tracking_Algorithms", dataName="Lagrangian_UpdraftTracking",
                                dtype='float32',codeSection = "Project_Algorithms")

In [ ]:
#data manager class (for saving data)
DataManager_TrackedProfiles = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="Tracked_Profiles", dataName="Tracked_Profiles",
                                dtype='float32',codeSection = "Project_Algorithms")

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","2_Tracking_Algorithms"))
from CLASSES_TrackingAlgorithms import TrackingAlgorithms_DataLoading_Class, Results_InputOutput_Class, TrackedParcel_Loading_Class

In [ ]:
# IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","3_Tracked_Profiles"))
from CLASSES_TrackedProfiles import TrackedProfiles_DataLoading_CLASS, TrackedProfiles_Plotting_CLASS, LocationSubset_Plotting_CLASS

In [ ]:
##############################################
#SETUP

In [ ]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
def MakeDataDictionary(variableNames,t,printstatement=False):
    timeString = ModelData.timeStrings[t]
    # print(f"Getting data from {timeString}","\n")
    
    dataDictionary = {variableName: CallLagrangianArray(ModelData, DataManager, timeString, variableName=variableName, printstatement=printstatement) 
                      for variableName in variableNames}      
    return dataDictionary
    
def GetSpatialData(t):    
    variableNames = ['Z']
    dataDictionary = MakeDataDictionary(variableNames,t)
    [Z] = (dataDictionary[k] for k in variableNames)
    return Z

In [ ]:
##############################################
#DATA LOADING

In [ ]:
#Loading in Tracked Parcels Info
trackedArrays,LevelsDictionary = TrackedParcel_Loading_Class.LoadingSubsetParcelData(ModelData,DataManager,
                                                         Results_InputOutput_Class)
hLines,hLineColors = TrackedProfiles_Plotting_CLASS.GetHLines(LevelsDictionary)

In [ ]:
##############################################
#PLOTTING FUNCTIONS FOR VARIABLES

In [ ]:
def ExtraSingleAxisModifications(variableName, axis):
    if variableName in ['W',"VMF_g","VMF_c","HMC"]:
        axis.axvline(0,color='gray',linestyle='dashed',zorder=-15)
    if variableName in ['HMC']:
        apply_scientific_notation([axis],decimals=2)

def ExtraAllAxisModifications(axes,variableNames):

    #snap axises to limits
    SetEvenTicks(axes, dim='x')

    #matching W and VMF x axises
    index1=variableNames.index('W')
    index2=variableNames.index('VMF_g')
    MatchAxisLimits([axes[index1], axes[index2]], dim='x')

    index=variableNames.index('QCQI')
    axes[index].set_xlim(left=0)

    #RH_vapor xlim max 100%
    index=variableNames.index('RH_vapor')
    axes[index].set_xlim(right=100)

In [ ]:
def PlotAllVariables(profiles, profilesSE, variableNames, variableInfo,
                     parcelTypes, parcelDepths, ncols=3, figsize=(15, 12),
                     locationSubset=""):

    n_vars = len(variableNames)
    nrows = int(np.ceil(n_vars / ncols))

    fig = plt.figure(figsize=figsize)
    gs = gridspec.GridSpec(nrows, ncols, figure=fig, wspace=0.2, hspace=0.35)

    axes = [fig.add_subplot(gs[i // ncols, i % ncols]) for i in range(n_vars)]

    for i, var in enumerate(variableNames):
        axis = axes[i]
        TrackedProfiles_Plotting_CLASS.PlotSingleVariable(axis, profiles,
                                                          profilesSE, var,
                                                          variableInfo,
                                                          parcelTypes,
                                                          parcelDepths,
                                                          hLines,hLineColors,
                                                          locationSubset=locationSubset)
        ExtraSingleAxisModifications(var, axis)

    # Turn off unused axes (if any)
    for j in range(len(variableNames), nrows * ncols):
        fig.add_subplot(gs[j // ncols, j % ncols]).axis("off")

    TrackedProfiles_Plotting_CLASS.AddCategoryLegend(fig, parcelTypes)
    TrackedProfiles_Plotting_CLASS.AddDepthLegend(axes[0])

    ExtraAllAxisModifications(axes,variableNames)

    return fig

In [ ]:
##############################################
#DATA LOADING FOR VARIABLES

In [ ]:
dataName = "Variables"
trackedProfileArrays = TrackedProfiles_DataLoading_CLASS.LoadProfile(ModelData,DataManager_TrackedProfiles, dataName, t='combined')

trackedProfiles_SE = TrackedProfiles_DataLoading_CLASS.ExtractProfileStandardErrorArrays(trackedProfileArrays,ProfileStandardError)

In [ ]:
##############################################
#PLOTTING SETUP FOR VARIABLES

In [ ]:
variableInfo = {
    "QV": {
        "label": r"$q_v$",
        "units": r"($g\ kg^{-1}$)",
        "multiplier": 1e3
    }, 
    "QCQI": {
        "label": r"$q_c+q_i$",
        "units": r"($g\ kg^{-1}$)",
        "multiplier": 1e3
    }, 
    "RH_vapor": {
        "label": r"$RH_v$",
        "units": "(%)",
        "multiplier": 1e2
    }, 
    "W": {
        "label": "w",
        "units": r"($m\ s^{-1}$)",
        "multiplier": 1
    },
    "VMF_g": {
        "label": r"$VMF$",
        "units": r"($kg\ m^{-2}\ s^{-1}$)",
        "multiplier": 1
    },
    "HMC": {
        "label": "HMC",
        "units": r"($g\ kg^{-1}\ s^{-1}$)",
        "multiplier": 1e3
    },    
    "THETA_v": {
        "label": r"$\theta_v$",
        "units": "(K)",
        "multiplier": 1
    },
    "THETA_e": {
        "label": r"$\theta_e$",
        "units": "(K)",
        "multiplier": 1
    },
    "MSE": {
        "label": r"$MSE$",
        "units": "(K)",
        "multiplier": 1/1005.7
    }
}

In [ ]:
##############################################
#PLOTTING FOR VARIABLES

In [ ]:
variableNames = ["QV", "QCQI","RH_vapor","W","VMF_g", "HMC","THETA_v","THETA_e","MSE"]

fig = PlotAllVariables(
    profiles=trackedProfileArrays,
    profilesSE=trackedProfiles_SE,
    variableNames=variableNames,
    variableInfo=variableInfo,
    parcelTypes=["CL", "nonCL", "SBF"],
    parcelDepths=["SHALLOW", "DEEP"],
    locationSubset="")

#saving
fileName=f"Tracked_Profiles_{dataName}" 
# SaveFigure(fig,plotType=f"Project_Algorithms/Tracked_Profiles/Tracked_Profiles_{dataName}",fileName=fileName)

In [ ]:
##############################################
#PLOTTING FUNCTIONS FOR LOCATION SUBSETTING

In [ ]:
def ApplyFormatting(fig, variableNames):
    axes = fig.get_axes()
    SetEvenTicks(axes)
    apply_scientific_notation(axes, decimals=1)

    # Match W + VMF_g together
    lst = []
    for variableName in ["W", "VMF_g"]:
        variableAxes = LocationSubset_Plotting_CLASS.GetVariableAxes(fig, variableName)
        lst.extend(variableAxes)
    MatchAxisLimits_V2(lst)

    # Match each variable’s axes across parcel types
    for variableName in variableNames:
        variableAxes = LocationSubset_Plotting_CLASS.GetVariableAxes(fig, variableName)
        MatchAxisLimits_V2(variableAxes)
    

In [ ]:
##############################################
#PLOTTING FOR LOCATION SUBSETTING

In [ ]:
variableNames = ["QV", "QCQI","RH_vapor","W","VMF_g", "HMC","THETA_v","THETA_e","MSE"]

fig = PlotAllVariables(
    profiles=trackedProfileArrays,
    profilesSE=trackedProfiles_SE,
    variableNames=variableNames,
    variableInfo=variableInfo,
    parcelTypes=["CL", "nonCL", "SBF"],
    parcelDepths=["SHALLOW", "DEEP"],
    locationSubset="_left")

#saving
fileName=f"Tracked_Profiles_{dataName}_LEFT" 
SaveFigure(fig,plotType=f"Project_Algorithms/Tracked_Profiles/Tracked_Profiles_{dataName}",fileName=fileName)

In [ ]:
variableNames = ["QV", "QCQI","RH_vapor","W","VMF_g", "HMC","THETA_v","THETA_e","MSE"]

fig = PlotAllVariables(
    profiles=trackedProfileArrays,
    profilesSE=trackedProfiles_SE,
    variableNames=variableNames,
    variableInfo=variableInfo,
    parcelTypes=["CL", "nonCL", "SBF"],
    parcelDepths=["SHALLOW", "DEEP"],
    locationSubset="_right")

#saving
fileName=f"Tracked_Profiles_{dataName}_RIGHT" 
SaveFigure(fig,plotType=f"Project_Algorithms/Tracked_Profiles/Tracked_Profiles_{dataName}",fileName=fileName)

In [ ]:
variableNames = ["QV", "QCQI","RH_vapor","W","VMF_g", "HMC","THETA_v","THETA_e","MSE"]
fig = LocationSubset_Plotting_CLASS.PlotProfiles(
    trackedProfileArrays,
    variableInfo,
    variableNames = variableNames)
ApplyFormatting(fig, variableNames)

#saving
fileName=f"Tracked_Profiles_{dataName}_LocationSubset" 
SaveFigure(fig,plotType=f"Project_Algorithms/Tracked_Profiles/Tracked_Profiles_{dataName}",fileName=fileName)